### Part 2 Leaf Recognition Using CNN with Image Generator
    Train the csv file and the image file at the same time. change the pixel and train method and find the most suitable model.

In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize, rotate, SimilarityTransform, warp
from skimage.filters import sobel
import glob


C:\Users\lanwe\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\lanwe\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import gc
gc.collect()

np.random.seed(1337) # for reproducibility


### Loading data

In [3]:
# Loading the train.csv to find features for each training point
train = pd.read_csv('100 leaves plant species/train0.csv', usecols=['id', 'species'])
mtrain = train.shape[0]
test = pd.read_csv('100 leaves plant species/test01.csv', usecols = [0])
mtest = test.shape[0]
#print(test.head())

### Resize and read images
Fill image to same size

In [4]:
print('Loading and resizing images...')# % len(image_paths))
img_rows, img_cols = 64, 64 #96,96 #40, 40
output_shape = (img_rows, img_cols)

train_images = np.zeros((mtrain, img_rows, img_cols))
for i in range(mtrain):
    #image = imread('images/'+str(train.id.iloc[i])+'.jpg')
    image = imread('100 leaves plant species/image/'+str(train.id.iloc[i])+'.jpg')
    rimage = resize(image, output_shape=output_shape)
    train_images[i] = sobel(rimage)
    
test_images = np.zeros((mtest, img_rows, img_cols))
for i in range(mtest):
    image = imread('100 leaves plant species/image/'+str(test.id.iloc[i])+'.jpg')
    rimage = resize(image, output_shape=output_shape)
    test_images[i] = sobel(rimage)

print('Train images shape: {}'.format(train_images.shape)) # 1240
print('Test images shape: {}'.format(test_images.shape)) # 357

Loading and resizing images...


C:\Users\lanwe\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Train images shape: (1240, 64, 64)
Test images shape: (357, 64, 64)


Encode

In [5]:
# Target
le = LabelEncoder()
target = le.fit_transform(train.species)
print("Target shape: {}".format(target.shape))
print(np.unique(target).size)
print(target[:5])

Target shape: (1240,)
100
[0 0 0 0 0]


### Bigger validation set
change the train set as validation set so that the validation set is the biggest one.

In [6]:
# Create random train and validation sets out of 20% samples
#Xtrain, Xval, ytrain, yval = train_test_split(train_images, target, test_size=0.15,
#                                           stratify=target, random_state=14) #11)
Xtrain, ytrain = train_images, target
Xval, yval = Xtrain, ytrain 
print('\nXtrain, ytrain shapes ' + str((Xtrain.shape, ytrain.shape)))
print('Xval, yval shapes ' + str((Xval.shape, yval.shape)))


Xtrain, ytrain shapes ((1240, 64, 64), (1240,))
Xval, yval shapes ((1240, 64, 64), (1240,))


### Reshape data as a 4-dim tensor [number samples, width, height, color channels]

In [7]:
print('Reshape as 4-dim tensor (Tensorflow backend)')
Xtrain = Xtrain.reshape(Xtrain.shape[0], img_rows, img_cols, 1) # 792
Xval = Xval.reshape(Xval.shape[0], img_rows, img_cols, 1) # 198
Xtest = test_images.reshape(test_images.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
print(Xtrain.shape, Xval.shape, Xtest.shape)


Reshape as 4-dim tensor (Tensorflow backend)
(1240, 64, 64, 1) (1240, 64, 64, 1) (357, 64, 64, 1)


### Image batch generator
reduce the occupication of GPU

In [8]:
def imageGenerator(X, y, batch_size):
    img_rows, img_cols = X.shape[1], X.shape[2]
    resc = 0.02
    rot = 5
    transl = 0.01*img_rows
    while 1: # Infinite loop
        batchX = np.zeros((batch_size, img_rows, img_cols, 1))
        # batch_size random indices over train images
        batch_ids = np.random.choice(X.shape[0], batch_size)
        for j in range(batch_ids.shape[0]): # Loop over random images
            # Rotate around center
            imagej = rotate(X[batch_ids[j]], angle =rot*np.random.randn())
            # Rescale and translate
            tf = SimilarityTransform(scale = 1 + resc*np.random.randn(1,2)[0],
                                translation = transl*np.random.randn(1,2)[0]) 
            batchX[j] = warp(imagej, tf)
        #batchX = np.reshape(batchX, (batch_size,-1)) # Flattened images for FNN
        #print(batchX.shape, y[batch_ids].shape)
        yield (batchX, y[batch_ids])
        

### Set parameter

In [9]:
# Convolutional Neural Network
print('Train convolutional neural network')
# Parameters
# Batch size: number of training examples in one forward/backward pass. 
# The higher the batch size, the more memory space you'll need
batch_size = 32
# Epoch: one forward pass and one backward pass of all the training examples
nb_epoch = 10
n_extension = 10
samples_per_epoch = batch_size*(n_extension*Xtrain.shape[0] // batch_size)
# number of convolutional filters to use (a hidden layer is segmented into feature maps
# where each unit in a feature map looks for the same feature but at different positions of the input image)
#nb_filters = 64
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
# Number of classes
nb_classes = np.unique(ytrain).size
#kernel_size = (5, 5)
print('Model Parameters:')
print('Batch size: %d, epochs: %d, samples per epoch: %d, n classes: %d' % 
                                (batch_size, nb_epoch,samples_per_epoch, nb_classes))

print('\nTraining Keras Convolutional Neural Network...')
# Convert class vectors to binary class matrices (one-hot encoder)
ytrain = np_utils.to_categorical(ytrain, nb_classes)
yval = np_utils.to_categorical(yval, nb_classes)

Train convolutional neural network
Model Parameters:
Batch size: 32, epochs: 10, samples per epoch: 12384, n classes: 100

Training Keras Convolutional Neural Network...


### Train and test

In [10]:
# Create model
model = Sequential()
# Add hidden layers
# Conv2D layer with 5x5 kernels (local weights) and 32 conv filters 
# (or feature maps), expects 2d images as inputs
model.add(Convolution2D(16, 5, 5,  border_mode='valid', input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5)) # Regularization method, exclude 50% units
# Another conv2D layer
model.add(Convolution2D(32, 5, 5))
model.add(Activation('relu'))
# Pool2D layer, a form of non-linear down-sampling to prevent
# overfitting and provide a form of translation invariance
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25)) # Regularization method, exclude 25% units
# Flattenig layer, converts 2D matrix into vectors
model.add(Flatten())
# Standard fully connected layer with 128 units
# model.add(Dense(256))
# model.add(Dropout(0.25)) # Regularization method, exclude 25% units
# model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))

# Output layer
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
# Fit model with generator
model.fit_generator(imageGenerator(Xtrain, ytrain, batch_size), 
                    samples_per_epoch = samples_per_epoch,
                    nb_epoch=nb_epoch, verbose=1, validation_data=(Xval, yval))
#model.fit(Xtrain, ytrain, batch_size=batch_size, nb_epoch=nb_epoch,
#          verbose=1, validation_data=(Xval, yval))
score = model.evaluate(Xval, yval, verbose=0)
print('Validation loss: %0.5f' % score[0])
print('Validation accuracy: %0.2f' % (100*score[1]))

C:\Users\lanwe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), padding="valid", input_shape=(64, 64, 1...)`
  
C:\Users\lanwe\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5))`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\lanwe\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\lanwe\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `fit_generator` call to th

Epoch 1/10
12384/12384 [==============================] - 1577s 127ms/step - loss: 0.2460 - acc: 0.9261 - val_loss: 0.0027 - val_acc: 0.9992
Epoch 2/10
12384/12384 [==============================] - 1522s 123ms/step - loss: 0.0507 - acc: 0.9843 - val_loss: 0.0074 - val_acc: 0.9984
Epoch 3/10
12384/12384 [==============================] - 1509s 122ms/step - loss: 0.0357 - acc: 0.9891 - val_loss: 1.7023e-04 - val_acc: 1.0000
Epoch 4/10
12384/12384 [==============================] - 1509s 122ms/step - loss: 0.0292 - acc: 0.9913 - val_loss: 3.6244e-04 - val_acc: 1.0000
Epoch 5/10
12384/12384 [==============================] - 1510s 122ms/step - loss: 0.0257 - acc: 0.9924 - val_loss: 0.0024 - val_acc: 0.9992
Epoch 6/10
12384/12384 [==============================] - 1513s 122ms/step - loss: 0.0232 - acc: 0.9932 - val_loss: 9.5909e-05 - val_acc: 1.0000
Epoch 7/10
12384/12384 [==============================] - 1512s 122ms/step - loss: 0.0217 - acc: 0.9939 - val_loss: 1.9166e-04 - val_acc: 1.00

we found that this one is the best one. When we use the image and the csv file at the same time, our accuracy is almost 1. But we found that the dimension is too high, so the speed of the process is too slow. In that case, we want to use some methods to reduce the dimension and we can do the machine learning at the same time. Then we found 2 methods, PCA and k-means to do it.

Tensorflow CNN：https://www.kaggle.com/jiexus/cnn-with-tensorflow/notebook
               
               https://keras.io/preprocessing/image/
               
               https://blog.csdn.net/sinat_26917383/article/details/74922230

 Copyright <2018> <Weiyi Lan YiQun Xu Yang Zong>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.